# Random fields

In [18]:
import sys, os, json
cwd = os.getcwd()
if cwd not in sys.path:
    sys.path.append(cwd)
from sklearn.model_selection import train_test_split
import pycrfsuite
#pip install python-crfsuite

#c'est dans le dossier utils que j'ai mis un fichier cleaning.py qui contient mes fonctions
from utils.cleaning import alternative_data_base, extract_features, get_entities
dbs_path = 'query_dbs/'
list_files = os.listdir(dbs_path)
# je liste les files dans la directory query_dbs
print(list_files)
list_query_type = [file_name.replace('.json','') for file_name in list_files]
# je liste les différents type de query
print(list_query_type)

['PlayMusic.json', 'BookRestaurant.json', 'AddToPlaylist.json', 'GetWeather.json', 'SearchCreativeWork.json', 'AllQueries.json', 'SearchScreeningEvent.json', 'RateBook.json']
['PlayMusic', 'BookRestaurant', 'AddToPlaylist', 'GetWeather', 'SearchCreativeWork', 'AllQueries', 'SearchScreeningEvent', 'RateBook']


In [21]:
number = 0
type_of_query = list_query_type[number]
data = json.load(open(dbs_path  + type_of_query+ '.json'))[type_of_query]
data = alternative_data_base(data)
X = [extract_features(query) for query in data]
y = [get_entities(query) for query in data]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
trainer = pycrfsuite.Trainer(verbose=True)
# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)
# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,
    # coefficient for L2 penalty
    'c2': 0.01,  
    # maximum number of iterations
    'max_iterations': 200,
    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})
# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

In [ ]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 10
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))